## Install MySQLClient

In [1]:
!pip install mysqlclient

## Connect to database
Credentials are provided by server owner. Accounts only have `SELECT` privileges on the authorized database.

In [2]:
import MySQLdb
from IPython.display import clear_output # clear credentials from persisting

In [3]:
username = input('username: ')
password = input('password: ')
clear_output()

In [4]:
db = MySQLdb.connect(host='sansa.cs.uoregon.edu', 
                     port=3331, 
                     user=username, 
                     password=password, 
                     db='ideas_db', 
                     charset='utf8', 
                     use_unicode=True)

### Disconnect from database
When done, be sure to call `db.close()` to safely terminate the connection.

## Database information

TODO: Replace with images of model?

*MySQLdb Documentation*: https://mysqlclient.readthedocs.io/user_guide.html

### Table 1: project
1. id
2. source_url
3. name
4. last_updated	
5. fork_of_id

### Table 2: author
1. id
2. username
3. email

### Table 3*: project_has_author
1. id
2. author_id
3. project_id

### Table 4: commit
1. id
2. datetime
3. author_id
4. project_id
5. message
6. hash
7. branch

### Table 5: diff
1. id
2. file_path
3. language
4. commit_id
5. body

---

***Not yet implemented tables, subject to change***

### Table 6: person
1. id
2. alias
3. email
4. github_username
5. gitlab_username

### Table 7*: person_has_author
1. id
2. author_id
3. person_id

### Table 8: issue

### Table 9: issue_comment

### Table 10: pull_request

\* Bridge table

## Cursor object
To execute queries, create a cursor. Similar to the connection, be sure to call `cursor.close()` when you are done. A cursor can execute multiple queries so no need to create one for each query.

In [5]:
cursor = db.cursor()

## Example 1: Viewing projects (and other tables)

In [6]:
import pandas as pd

In [7]:
query = 'select * from project' # MySQL keywords are case-insensitive
cursor.execute(query) # Run the query
data = cursor.fetchall() # alternatively, use fetchone(), fetchmany(n)
pd.DataFrame(data)

,0,1,2,3,4
0,9,https://github.com/lammps/lammps.git,lammps,2021-02-24 01:48:15,None
1,10,https://github.com/spack/spack.git,spack,2021-02-24 17:06:06,None


If you want to include column names of a single table run the following query (though it might be easier to just manually write them):

In [8]:
def get_columns(table):
    query = """SELECT `COLUMN_NAME` 
FROM `INFORMATION_SCHEMA`.`COLUMNS` 
WHERE `TABLE_SCHEMA`='ideas_db' 
    AND `TABLE_NAME`=%s"""
    cursor.execute(query, (table,)) # make sure args are tuples
    columns = cursor.fetchall()
    return columns

In [9]:
columns = get_columns('project')
pd.DataFrame(data, columns=columns)

,id,source_url,name,last_updated,fork_of_id
0,9,https://github.com/lammps/lammps.git,lammps,2021-02-24 01:48:15,None
1,10,https://github.com/spack/spack.git,spack,2021-02-24 17:06:06,None


## Example 2: Viewing authors associated with a project

In [10]:
url = 'https://github.com/lammps/lammps.git'
query = '''select username, email 
from author a join project_has_author pha on(a.id = pha.author_id) join project p on(pha.project_id = p.id)
where source_url = %s
'''
cursor.execute(query, (url,))
data = cursor.fetchall()
columns = ['username', 'email']
pd.DataFrame(data, columns=columns)

,username,email
0,Axel Kohlmeyer,akohlmey@gmail.com
1,Richard Berger,richard.berger@temple.edu
2,jrgissing,jrgiss05@gmail.com
3,Stan Moore,stamoor@sandia.gov
4,Michael Brown,michael.w.brown@intel.com
...,...,...
258,pscrozi,pscrozi@f3b2605a-c512-4ea7-a41b-209d697bcdaa
259,jatempl,jatempl@f3b2605a-c512-4ea7-a41b-209d697bcdaa
260,rjones,rjones@f3b2605a-c512-4ea7-a41b-209d697bcdaa
261,jzimmer,jzimmer@f3b2605a-c512-4ea7-a41b-209d697bcdaa


## Example 3: Viewing commits by an author in a project

In [11]:
url = 'https://github.com/lammps/lammps.git'
name = 'Axel Kohlmeyer'
query = '''select c.datetime, c.message, c.hash, c.branch
from commit c join author a on(c.author_id = a.id) join project p on(c.project_id = p.id)
where a.username = %s and p.source_url = %s'''
cursor.execute(query, (name, url,))
data =cursor.fetchall()
columns = ['datetime', 'message', 'hash', 'branch']
pd.DataFrame(data, columns=columns)

,datetime,message,hash,branch
0,2021-02-22 04:05:20,need to use column 1 in fix ave/time example,f3ee948450bbc63eef80c3bad1a2de382dda7b75,* master\n remotes/origin/HEAD -> origin/mast...
1,2021-02-21 17:40:11,must explicitly register fix gpu,4786391fad023faaa676de5957cd3ea7d3ead24f,* master\n remotes/origin/HEAD -> origin/mast...
2,2021-02-21 14:11:38,correct library prefix name use also for build...,e0e89c588ba0c907f60824806dcf5ea84e8090d5,* master\n remotes/origin/HEAD -> origin/mast...
3,2021-02-21 13:42:30,update md5sum for updated archive (again),db95552f2b4c28c630c5c9a35b3a83b7892e990e,* master\n remotes/origin/HEAD -> origin/mast...
4,2021-02-21 13:21:45,correct how to construct the path to the gener...,1a68d761a3060cfd7c8ead56b495df070a847a23,* master\n remotes/origin/HEAD -> origin/mast...
...,...,...,...,...
5407,2016-08-21 08:17:18,fix bug in recent dump changes for -DLAMMPS_BI...,13836840484427e3bb9bee948cca77e694d03425,* master\n remotes/origin/HEAD -> origin/mast...
5408,2016-08-16 20:54:56,fix indexing bugs in accessing compute and fix...,9babb7a4c2ae3152554a8b7ea4458f286788a1e7,* master\n remotes/origin/HEAD -> origin/mast...
5409,2016-08-14 19:01:32,thread timing summare needs to be marked as pr...,c88e9b46cf976f6a72b2ec664c6079e0feda575d,* master\n remotes/origin/HEAD -> origin/mast...
5410,2016-08-13 07:53:29,correct small (but harmless) logic error.\n ...,730e3cb4ac9d805ef39ea2b99d03c83a5ac6bbab,* master\n remotes/origin/HEAD -> origin/mast...


## Example 4: Viewing commits between two dates in a project

In [12]:
import datetime

In [13]:
start = datetime.datetime(year=2020, month=11, day=1)
end = datetime.datetime(year=2020, month=12, day=1)

In [14]:
url = 'https://github.com/lammps/lammps.git'
query = '''select c.datetime, c.message, a.username
from commit c join author a on(c.author_id = a.id) join project p on(c.project_id = p.id)
where (c.datetime between %s and %s) and p.source_url = %s'''
cursor.execute(query, (start, end, url,))
data = cursor.fetchall()
columns = ['datetime', 'message', 'username']
pd.DataFrame(data, columns=columns)

,datetime,message,username
0,2020-11-27 12:17:39,fix whitespace,Axel Kohlmeyer
1,2020-11-27 12:16:32,fix typos and remove false false positive,Axel Kohlmeyer
2,2020-11-27 12:16:04,remove unbundled potential file (again),Axel Kohlmeyer
3,2020-11-25 12:42:37,complete first draft of unit test documentation,Axel Kohlmeyer
4,2020-11-25 12:03:14,whitespace (again),Axel Kohlmeyer
...,...,...,...
261,2020-11-02 13:39:13,fix bug with memory allocation,iafoss
262,2020-11-02 13:35:50,fix bug with memory allocation\n \n fix ...,iafoss
263,2020-11-02 13:24:57,"Revert ""Add files via upload""\n \n This ...",iafoss
264,2020-11-02 13:12:57,Add files via upload,iafoss


## Example 5: Viewing diffs for a commit

In [15]:
url = 'https://github.com/lammps/lammps.git'
hash = '2a6561e52af01c82106396b401ad6e19a761b31d'
query = '''select c.hash, d.file_path, d.language, d.body
from diff d join commit c on(d.commit_id = c.id) join project p on (c.project_id = p.id)
where p.source_url = %s and c.hash = %s'''
cursor.execute(query, (url, hash,))
data = cursor.fetchall()
columns = ['hash', 'file_path', 'language', 'body']
pd.DataFrame(data, columns=columns)

,hash,file_path,language,body
0,2a6561e52af01c82106396b401ad6e19a761b31d,examples/USER/tally/in.force,PLACEHOLDER,+# make certain that shake constraints are sat...
1,2a6561e52af01c82106396b401ad6e19a761b31d,examples/USER/tally/in.pe,PLACEHOLDER,+# make certain that shake constraints are sat...


In [16]:
# Don't forget!
cursor.close()
db.close()